# ENUM_BANK의 주소를 bloomberg 에서 가져오는 Crawler

## 필요한 라이브러리들 인젝션

In [86]:
from selenium import webdriver
from bs4 import BeautifulSoup
import json
import pandas as pd
import re
import time

## 셀레늄에 헤더정보 설정 및 헤더리스 설정

In [210]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('headless')
chrome_options.add_argument('window-size=1920x1080')
chrome_options.add_argument("disable-gpu")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36")
chrome_options.add_argument("lang=ko_KR") # 한국어!
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)

## GIVEN
### bing에서 bloomberg company profile + 기업명 검색결과 주소중 첫번째 검색결과 링크를 블룸버그경로로 설정

In [87]:
df = pd.read_csv('enum_bank_japan - Sheet 1.csv')
df_enum_bank_japan = df
target_list = df_enum_bank_japan.descr
url_origin1 = 'https://www.bing.com/search?q='
bloomberg_company_profile = ' bloomberg company profile'
tmp_url = []
for bank in target_list:
    url1 = url_origin1 + bank + bloomberg_company_profile
    tmp_url.append(url1)
bloomber_url = []
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
for url in tmp_url :
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    bloomber_url.append(soup.find_all('a')[13].attrs['href'])
driver.close()
bloomber_url

### body > div:nth-child(9) > div > div > div.container.m-t-5.m-b-5 > div > div > ul > li:nth-child(1) > small > span:nth-child(2)'

### body > div:nth-child(9) > div > div > div.container.m-t-5.m-b-5 > div > div > ul > li:nth-child(1) > div > a

In [233]:
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
driver.get('https://transferwise.com/gb/swift-codes/banks?country=0&city=0&name=THE+TOHOKU+BANK')
soup = BeautifulSoup(driver.page_source,'html.parser')

In [220]:
tmp_addr = soup.select('div div div div div div ul li small span')
print(tmp_addr[1].text)
tmp_bank = soup.select('div div div div div div ul li div a')
print(tmp_bank[0].text)

## WHEN1
### 해당 기업의 블룸버그 주소에서 address 정보를 가져옴

In [217]:
addr = []
addr_tmp = []


# driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver')
for idx, url in enumerate(bloomber_url) :
    print(df_enum_bank_japan.descr[idx] , idx, url)
    driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    try :
        mydivs1 = soup.find("meta", {"itemprop": "address"})['content']
        if (mydivs1 is not None) or (len(mydivs1) != 0) :
            addr.append(mydivs1)
            print(mydivs1)
        else :
            mydivs3 = soup.find_all("div", {"itemprop": "address"})
            if (mydivs3 is not None) or (len(mydivs3) != 0)  :
                addr.append(mydivs3)
                print(mydivs3)
            else :
                mydivs2 = soup.find_all("div", {"class": "infoTableItemValue__e188b0cb"})[4]
                print (mydivs2)
                addr.append(mydivs2)
                
    except Exception as ex: # 에러시 해당 주소 반환
        print(ex)
        addr.append(url)
    driver.close()
    time.sleep(20)
    if idx > 10 : 
        break

SBI SUMISHIN NET BANK, LTD. 0 https://www.bloomberg.com/profiles/companies/0186804D:JP-sbi-sumishin-net-bank-ltd
'NoneType' object is not subscriptable
AEON BANK, LTD. 1 https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd
'NoneType' object is not subscriptable
THE HOKKAIDO BANK , LTD. 2 https://www.bloomberg.com/profiles/companies/8353:JP-hokkaido-bank-ltd/the
'NoneType' object is not subscriptable
THE AOMORI BANK , LTD. 3 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=876065
'NoneType' object is not subscriptable
THE MICHINOKU BANK , LTD. 4 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=879235
'NoneType' object is not subscriptable
THE AKITA BANK , LTD. 5 https://www.bloomberg.com/research/stocks/snapshot/snapshot.asp?capid=876216
'NoneType' object is not subscriptable
THE HOKUTO BANK , LTD. 6 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=5533401
'NoneType' object is not subscriptable
THE S

KeyboardInterrupt: 

In [251]:
addr = []
bank = []
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
url = 'https://transferwise.com/gb/swift-codes/banks?country=0&city=0&name='
descr_list = df_enum_bank_japan.descr
# driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver')
for idx, df_data in enumerate(descr_list) :
#     print(idx)
    tmp_str = url + df_data
    driver.get(tmp_str)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    tmp_bank = soup.select('div div div div div div ul li div a')
    tmp_addr = soup.select('div div div div div div ul li small span')
    bank.append(tmp_bank[0].text)
    addr.append(tmp_addr[1].text)
#     print (tmp_bank[0].text , " : ", tmp_addr[1].text)


In [252]:
result = []
for idx, v in enumerate(addr) :
    print(bank[idx], '\n', df_enum_bank_japan.descr[idx], "\n\t",addr[idx])

Sbi Sumishin Net Bank, Ltd. 
 SBI SUMISHIN NET BANK, LTD. 
	 Izumi Garden Tower Tokyo 1066018, 
Ping An Bank Co., Ltd. (formerly Shenzhen Development Bank Co.,ltd. 
 AEON BANK, LTD. 
	 Ping an Bank Co.,Ltd.,Zheng Zhou Br Zhengzhou, 
Hokkaido Bank, Ltd., The 
 THE HOKKAIDO BANK , LTD. 
	 1 Nishi 4-Chome Odori, Sapporo 060-8676, 
Aomori Bank Ltd., The 
 THE AOMORI BANK , LTD. 
	 9-30 Hashimoto 1-Chome Aomori 030-0823, 
Michinoku Bank, Ltd., The 
 THE MICHINOKU BANK , LTD. 
	 1-3-1 Katta Aomori 030-8622, 
Akita Bank, Ltd.,The 
 THE AKITA BANK , LTD. 
	 2-1, Sanno 3-Chome Akita 010-8655, 
Hokuto Bank, Ltd., The 
 THE HOKUTO BANK , LTD. 
	 1-41 Nakadori 3-Chome Akita 010-0001, 
Ping An Bank Co., Ltd. (formerly Shenzhen Development Bank Co.,ltd. 
 THE SHONAI BANK , LTD. 
	 Ping an Bank Co.,Ltd.,Zheng Zhou Br Zhengzhou, 
Yamagata Bank, Ltd., The 
 THE YAMAGATA BANK , LTD. 
	 1-2, Nanokamachi, 3-Chome Yamagata 990-8642, 
Bank Of Iwate Ltd., The 
 THE BANK OF IWATE , LTD. 
	 2-3, 1-Chome, Chuo 

In [197]:
addr = []
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
driver.get( 'https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd')
soup = BeautifulSoup(driver.page_source,'html.parser')

mydivs1 = soup.find("meta", {"itemprop": "address"})['content']

if (mydivs1 is not None) or (len(mydivs1) != 0) :
    addr.append(mydivs1)
else : 
    mydivs2 = soup.find_all("div", {"class": "infoTableItemValue__e188b0cb"})[4]
    addr.append(mydivs2)

driver.close()
addr

['1-6-1 Roppongi, Minato-ku, Tokyo, 106-6018, Japan',
 'https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd',
 'https://www.bloomberg.com/profiles/companies/8353:JP-hokkaido-bank-ltd/the',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=876065',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=879235']

In [192]:
addr = []

driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
for idx, url in enumerate(bloomber_url) :
    print(idx, url)
    addr.append(url)
driver.close()
addr

0 https://www.bloomberg.com/profiles/companies/0186804D:JP-sbi-sumishin-net-bank-ltd
1 https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd
2 https://www.bloomberg.com/profiles/companies/8353:JP-hokkaido-bank-ltd/the
3 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=876065
4 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=879235
5 https://www.bloomberg.com/research/stocks/snapshot/snapshot.asp?capid=876216
6 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=5533401
7 https://www.bloomberg.com/profiles/companies/8347:JP-shonai-bank-ltd/the
8 https://www.bloomberg.com/profile/company/8344:JP
9 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=878782
10 https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=881271
11 https://www.bloomberg.com/profiles/people/15413837-kenichi-hotta
12 https://www.bloomberg.com/profile/company/8346:JP
13 https://www.bloomberg.com

['https://www.bloomberg.com/profiles/companies/0186804D:JP-sbi-sumishin-net-bank-ltd',
 'https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd',
 'https://www.bloomberg.com/profiles/companies/8353:JP-hokkaido-bank-ltd/the',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=876065',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=879235',
 'https://www.bloomberg.com/research/stocks/snapshot/snapshot.asp?capid=876216',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=5533401',
 'https://www.bloomberg.com/profiles/companies/8347:JP-shonai-bank-ltd/the',
 'https://www.bloomberg.com/profile/company/8344:JP',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=878782',
 'https://www.bloomberg.com/research/stocks/private/snapshot.asp?privcapId=881271',
 'https://www.bloomberg.com/profiles/people/15413837-kenichi-hotta',
 'https://www.bloomberg.com/profile/company/8346:JP',
 'htt

## WHEN2
### 결과값

## 결과값 정규식으로 주소만 가져오기

In [5]:
pattern = '\[(.*?)\];'
match = re.search(pattern, soup.find_all('script')[15].text)
json.loads('[' + match.group(1) + ']')[3].get('cpRate')

10.38119757

In [204]:
addr = []
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
driver.get( 'https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd')
soup = BeautifulSoup(driver.page_source,'html.parser')

mydivs1 = soup.find("meta", {"itemprop": "address"})['content']

if (mydivs1 is not None) or (len(mydivs1) != 0) :
    addr.append(mydivs1)
else : 
    mydivs2 = soup.find_all("div", {"class": "infoTableItemValue__e188b0cb"})[4]
    addr.append(mydivs2)

driver.close()
addr

['1-9-6 Edagawa, Koto-Ku, 135-0051, Japan']

In [ ]:

mydivs1 = soup.find_all("div", {"class": "address"})
if (mydivs1 is None) or (len(mydivs1) == 0) :
    mydivs2 = soup.find_all("div", {"class": "infoTableItemValue__e188b0cb"})[4]
    addr.append(mydivs2)
else : 
    addr.append(mydivs1)

In [174]:
driver = webdriver.Chrome('/Users/arthur/pywork/chromedriver', options=chrome_options)
driver.get('https://www.bloomberg.com/profiles/companies/AEONBZ:JP-aeon-bank-ltd')
soup = BeautifulSoup(driver.page_source,'html.parser')
driver.close()

In [166]:

mydivs1 = soup.find("meta", {"itemprop": "address"})['content']
if (mydivs1 is None) or (len(mydivs1) == 0) :
    mydivs2 = soup.find_all("div", {"class": "infoTableItemValue__e188b0cb"})[4]
    addr.append(mydivs2)
else : 
    addr.append(mydivs1)

IndexError: list index out of range